In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession
#Initialize Spark Session
spark = SparkSession.builder.appName("TestApp").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 03:45:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
findspark.find()

'/opt/anaconda3/lib/python3.11/site-packages/pyspark'

In [4]:
sc=spark.sparkContext

In [11]:
rdd = sc.parallelize([1,2,3,4])
rdd.collect()

[1, 2, 3, 4]

In [12]:
resultRdd = rdd.map(lambda a:a*2)
resultRdd.collect()

[2, 4, 6, 8]

In [16]:
resultRdd = rdd.flatMap(lambda x:(x,x*2))
resultRdd.collect()

[1, 2, 2, 4, 3, 6, 4, 8]

In [19]:
resultRdd = rdd.filter(lambda x: x%2!=0)
resultRdd.collect()

[1, 3]

In [24]:
rdd1 = sc.parallelize([(1,2),(2,3),(3,4),(1,5)])
resultRdd = rdd1.reduceByKey(lambda x,y: x+y)
resultRdd.collect()

[(2, 3), (1, 7), (3, 4)]

In [25]:
resultRdd = rdd1.groupByKey()
resultRdd.collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7f73e592b0d0>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f73e592b2d0>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f73e5923210>)]

In [26]:
for x,y in resultRdd.collect():
    print(f"Key:{x}, Values:{list(y)}")

Key:2, Values:[3]
Key:1, Values:[2, 5]
Key:3, Values:[4]


In [27]:
word_list = ["A","B","C","D","A","A","C"]

In [29]:
wordRdd = sc.parallelize(word_list)
wordRdd.collect()

['A', 'B', 'C', 'D', 'A', 'A', 'C']

In [41]:
countRdd = wordRdd.map(lambda x: (x,x)).groupByKey()
# countRdd.collect()

In [42]:
for key, val in countRdd.collect():
    print(f"{key} occurs {len(list(val))} times")

A occurs 3 times
D occurs 1 times
B occurs 1 times
C occurs 2 times


In [43]:
countRdd = wordRdd.map(lambda x: (x,1)).reduceByKey(lambda x,y:x+y)
for key, val in countRdd.collect():
    print(f"{key} occurs {(val)} times")

A occurs 3 times
D occurs 1 times
B occurs 1 times
C occurs 2 times


In [44]:
purchaseRdd = sc.textFile("/home/labuser/Downloads/purchases.csv")
purchaseRdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [57]:
testDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Downloads/purchases.csv")
testDf.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/21 08:59:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Downloads/purchases.csv


In [69]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
schema = StructType([
    StructField("",StringType(),True),
    StructField("apples",IntegerType(),True),
    StructField("oranges",IntegerType(),True),
])

In [70]:
testDf = spark.read.schema(schema).option("header",True).csv("/home/labuser/Downloads/purchases.csv")
testDf.show()

+------+------+-------+
|      |apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [71]:
testDf.printSchema()

root
 |-- : string (nullable = true)
 |-- apples: integer (nullable = true)
 |-- oranges: integer (nullable = true)



In [83]:
moviesDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Downloads/IMDB-Movie-Data.csv")
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [80]:
moviesSchema = StructType([
    StructField("Rank",IntegerType(),False),
    StructField("Title",StringType(),True),
    StructField("Genre",StringType(),True),
    StructField("Description",StringType(),True),
])

In [81]:
moviesDf = spark.read.schema(moviesSchema).option("header",True).csv("/home/labuser/Downloads/IMDB-Movie-Data.csv")
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)



In [84]:
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [85]:
from pyspark.sql.functions import *
moviesDf=moviesDf.withColumn("Rev_new",col("Revenue (Millions)"))
moviesDf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rev_new|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+-------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0| 333.13|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0

In [87]:
import datetime
moviesDf = moviesDf.withColumn("updated_timestamp",lit(datetime.datetime.now()))
moviesDf.show(2)

+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+--------------------+
|Rank|               Title|               Genre|         Description|    Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rev_new|   updated_timestamp|
+----+--------------------+--------------------+--------------------+------------+--------------------+----+-----------------+------+------+------------------+---------+-------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|  James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|            333.13|     76.0| 333.13|2023-09-21 10:55:...|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|Ridley Scott|Noomi Rapace, Log...|2012|              124|     7|485820|            126.46|     65.0| 126.46|2023-09-21 10:55:...|
+----

In [88]:
datetime.datetime.now()

datetime.datetime(2023, 9, 21, 10, 55, 22, 886725)

In [5]:
moviesDf = spark.read.option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/IMDB-Movie-Data.csv")

In [6]:
moviesDf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [27]:
moviesDf.describe()

DataFrame[summary: string, Rank: string, Title: string, Genre: string, Description: string, Director: string, Actors: string, Year: string, Runtime (Minutes): string, Rating: string, Votes: string, Rev_Million: string, Metascore: string]

In [21]:
testDf = moviesDf.repartition(10)

In [22]:
testDf.rdd.getNumPartitions()

10

In [20]:
testDf.write.csv("/home/labuser/Downloads/Output/Partitions")

In [24]:
sc.uiWebUrl

'http://ip-172-31-0-176.ap-south-1.compute.internal:4040'

In [25]:
print(f"{spark._jsc.sc().uiWebUrl().get()}/")

http://ip-172-31-0-176.ap-south-1.compute.internal:4040/


In [26]:
moviesDf.createOrReplaceTempView("Movies")

In [32]:
spark.sql("Select* from Movies").show(5)

+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+-----------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Rev_Million|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+----+-----------------+------+------+-----------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|2014|              121|   8.1|757074|     333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|2012|              124|     7|485820|     126.46|     65.0|
|   3|               Split|     Horror,Thriller|Three girls are k...|  M. Night Shyamalan|James McAvoy, Any...|2016|              117|   7.3|1576

In [33]:
spark.sql("Select Rank,Title from Movies where Rank>=1 and Rank<=4").show(5)

+----+--------------------+
|Rank|               Title|
+----+--------------------+
|   1|Guardians of the ...|
|   2|          Prometheus|
|   3|               Split|
|   4|                Sing|
+----+--------------------+



In [42]:
moviesDf.sort(col("Title").desc()).show(3)

+----+-----------+--------------------+--------------------+---------------+--------------------+----+-----------------+------+------+-----------+---------+
|Rank|      Title|               Genre|         Description|       Director|              Actors|Year|Runtime (Minutes)|Rating| Votes|Rev_Million|Metascore|
+----+-----------+--------------------+--------------------+---------------+--------------------+----+-----------------+------+------+-----------+---------+
|  75|   Zootopia|Animation,Adventu...|In a city of anth...|   Byron Howard|Ginnifer Goodwin,...|2016|              108|   8.1|296853|     341.26|     78.0|
| 432|Zoolander 2|              Comedy|Derek and Hansel ...|    Ben Stiller|Ben Stiller, Owen...|2016|              102|   4.7| 48297|      28.84|     34.0|
| 364| Zombieland|Adventure,Comedy,...|A shy student try...|Ruben Fleischer|Jesse Eisenberg, ...|2009|               88|   7.7|409403|      75.59|     73.0|
+----+-----------+--------------------+-------------------

In [48]:
testDf = moviesDf.select("Rank","Title","Rev_Million","Metascore")
testDf.show(3)

+----+--------------------+-----------+---------+
|Rank|               Title|Rev_Million|Metascore|
+----+--------------------+-----------+---------+
|   1|Guardians of the ...|     333.13|     76.0|
|   2|          Prometheus|     126.46|     65.0|
|   3|               Split|     138.12|     62.0|
+----+--------------------+-----------+---------+
only showing top 3 rows



In [54]:
from datetime import *
testDf = testDf.withColumn("last_updated_ts",lit(datetime.now()))
testDf.show(3)

+----+--------------------+-----------+---------+--------------------+
|Rank|               Title|Rev_Million|Metascore|     last_updated_ts|
+----+--------------------+-----------+---------+--------------------+
|   1|Guardians of the ...|     333.13|     76.0|2023-09-22 05:34:...|
|   2|          Prometheus|     126.46|     65.0|2023-09-22 05:34:...|
|   3|               Split|     138.12|     62.0|2023-09-22 05:34:...|
+----+--------------------+-----------+---------+--------------------+
only showing top 3 rows



In [69]:
testdf = testDf.selectExpr("Title","Rev_Million","Metascore","CAST(last_updated_ts AS DATE) AS Update_Ts")
testdf.show()

+--------------------+-----------+---------+----------+
|               Title|Rev_Million|Metascore| Update_Ts|
+--------------------+-----------+---------+----------+
|Guardians of the ...|     333.13|     76.0|2023-09-22|
|          Prometheus|     126.46|     65.0|2023-09-22|
|               Split|     138.12|     62.0|2023-09-22|
|                Sing|     270.32|     59.0|2023-09-22|
|       Suicide Squad|     325.02|     40.0|2023-09-22|
|      The Great Wall|      45.13|     42.0|2023-09-22|
|          La La Land|     151.06|     93.0|2023-09-22|
|            Mindhorn|     2490.0|     null|2023-09-22|
|  The Lost City of Z|       8.01|     78.0|2023-09-22|
|          Passengers|     100.01|     41.0|2023-09-22|
|Fantastic Beasts ...|     234.02|     66.0|2023-09-22|
|      Hidden Figures|     169.27|     74.0|2023-09-22|
|           Rogue One|     532.17|     65.0|2023-09-22|
|               Moana|     248.75|     81.0|2023-09-22|
|            Colossal|       2.87|     70.0|2023

In [59]:
testDf.select(col("Title").alias("MovieName")).show(3)
# testDf.show(3)

+--------------------+
|           MovieName|
+--------------------+
|Guardians of the ...|
|          Prometheus|
|               Split|
+--------------------+
only showing top 3 rows

+----+--------------------+-----------+---------+--------------------+
|Rank|               Title|Rev_Million|Metascore|     last_updated_ts|
+----+--------------------+-----------+---------+--------------------+
|   1|Guardians of the ...|     333.13|     76.0|2023-09-22 05:34:...|
|   2|          Prometheus|     126.46|     65.0|2023-09-22 05:34:...|
|   3|               Split|     138.12|     62.0|2023-09-22 05:34:...|
+----+--------------------+-----------+---------+--------------------+
only showing top 3 rows



In [71]:
testdf = testdf.withColumn("MovieName",col("Title"))
testdf.show()

+--------------------+-----------+---------+----------+--------------------+
|               Title|Rev_Million|Metascore| Update_Ts|           MovieName|
+--------------------+-----------+---------+----------+--------------------+
|Guardians of the ...|     333.13|     76.0|2023-09-22|Guardians of the ...|
|          Prometheus|     126.46|     65.0|2023-09-22|          Prometheus|
|               Split|     138.12|     62.0|2023-09-22|               Split|
|                Sing|     270.32|     59.0|2023-09-22|                Sing|
|       Suicide Squad|     325.02|     40.0|2023-09-22|       Suicide Squad|
|      The Great Wall|      45.13|     42.0|2023-09-22|      The Great Wall|
|          La La Land|     151.06|     93.0|2023-09-22|          La La Land|
|            Mindhorn|     2490.0|     null|2023-09-22|            Mindhorn|
|  The Lost City of Z|       8.01|     78.0|2023-09-22|  The Lost City of Z|
|          Passengers|     100.01|     41.0|2023-09-22|          Passengers|

In [72]:
testdf.count()

1000

In [109]:
testDf = moviesDf.select("Rank","Title","Rev_Million","Metascore","Director","Year")
testDf.show(3)

+----+--------------------+-----------+---------+------------------+----+
|Rank|               Title|Rev_Million|Metascore|          Director|Year|
+----+--------------------+-----------+---------+------------------+----+
|   1|Guardians of the ...|     333.13|     76.0|        James Gunn|2014|
|   2|          Prometheus|     126.46|     65.0|      Ridley Scott|2012|
|   3|               Split|     138.12|     62.0|M. Night Shyamalan|2016|
+----+--------------------+-----------+---------+------------------+----+
only showing top 3 rows



In [86]:
print(testDf.count())
droppedDf = testDf.dropDuplicates(["Director","Year"])
print(droppedDf.count())

1000
987


In [110]:
testDf.withColumn("Combine",concat_ws(" || ",col("Title"),col("Director"))).show(3)

+----+--------------------+-----------+---------+------------------+----+--------------------+
|Rank|               Title|Rev_Million|Metascore|          Director|Year|             Combine|
+----+--------------------+-----------+---------+------------------+----+--------------------+
|   1|Guardians of the ...|     333.13|     76.0|        James Gunn|2014|Guardians of the ...|
|   2|          Prometheus|     126.46|     65.0|      Ridley Scott|2012|Prometheus || Rid...|
|   3|               Split|     138.12|     62.0|M. Night Shyamalan|2016|Split || M. Night...|
+----+--------------------+-----------+---------+------------------+----+--------------------+
only showing top 3 rows



In [83]:
testDf.withColumn("TotalScore",(col("Rev_Million")+col("Metascore"))).show(3)

+----+--------------------+-----------+---------+------------------+------------------+
|Rank|               Title|Rev_Million|Metascore|          Director|        TotalScore|
+----+--------------------+-----------+---------+------------------+------------------+
|   1|Guardians of the ...|     333.13|     76.0|        James Gunn|            409.13|
|   2|          Prometheus|     126.46|     65.0|      Ridley Scott|191.45999999999998|
|   3|               Split|     138.12|     62.0|M. Night Shyamalan|            200.12|
+----+--------------------+-----------+---------+------------------+------------------+
only showing top 3 rows



In [84]:
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Rev_Million: double (nullable = true)
 |-- Metascore: double (nullable = true)



In [87]:
testDf = moviesDf.select("Rank","Title","Rev_Million","Rating","Year")
testDf.show(3)

+----+--------------------+-----------+------+----+
|Rank|               Title|Rev_Million|Rating|Year|
+----+--------------------+-----------+------+----+
|   1|Guardians of the ...|     333.13|   8.1|2014|
|   2|          Prometheus|     126.46|     7|2012|
|   3|               Split|     138.12|   7.3|2016|
+----+--------------------+-----------+------+----+
only showing top 3 rows



In [91]:
testDf = testDf.withColumn("Rating_value",when(col("Rating")<=5,"Average").when((col("Rating")>5) & (col("Rating")<=8),"Good").when((col("Rating")>8) & (col("Rating")<=10),"Best").otherwise("NA"))

In [102]:
testDf.sort(col("Rating")).show()

+----+--------------------+-----------+------+--------------------+------------+
|Rank|               Title|Rev_Million|Rating|                Year|Rating_value|
+----+--------------------+-----------+------+--------------------+------------+
| 830|      Disaster Movie|      14.17|   1.9|                2008|     Average|
| 429|         Srpski film|    43648.0|   104|Srdjan 'Zika' Tod...|          NA|
| 984|       Let's Be Cops|   112729.0|   104|Jake Johnson, Dam...|          NA|
| 202| A Cure for Wellness|    12193.0|   146|Dane DeHaan, Jaso...|          NA|
|  43|Don't Fuck in the...|       null|   2.7|                2016|     Average|
| 872|Dragonball Evolution|       9.35|   2.7|                2009|     Average|
| 560|           Idiocracy|        6.6|  2006|          Mike Judge|          NA|
| 632|       The Interview|        6.6|  2014|       Evan Goldberg|          NA|
| 185|          Forushande|        8.0|  2016|      Asghar Farhadi|          NA|
| 604|     Kung Fu Panda 3| 

In [103]:
testDf.sort(col("Rating").desc()).show()

+----+--------------------+-----------+-----------------+--------------------+------------+
|Rank|               Title|Rev_Million|           Rating|                Year|Rating_value|
+----+--------------------+-----------+-----------------+--------------------+------------+
| 386|             Everest|     2015.0|Baltasar Kormákur| together with Sc...|          NA|
| 619|           Mr. Right|    30053.0|               95|Anna Kendrick, Sa...|          NA|
| 978|       Amateur Night|     2229.0|               92|Jason Biggs, Jane...|          NA|
|  55|     The Dark Knight|     533.32|                9|                2008|        Best|
|   8|            Mindhorn|     2490.0|               89|Essie Davis, Andr...|          NA|
| 969|             Wrecker|     1210.0|               83|Anna Hutchison, A...|          NA|
|  81|           Inception|     292.57|              8.8|                2010|        Good|
| 118|              Dangal|      11.15|              8.8|                2016|  

In [111]:
def concat_shell(column):
    return column+"_shell"

In [112]:
shell_add = udf(concat_shell,StringType())

In [113]:
testDf.withColumn("new_col",shell_add(col("Title"))).show(3)

+----+--------------------+-----------+---------+------------------+----+--------------------+
|Rank|               Title|Rev_Million|Metascore|          Director|Year|             new_col|
+----+--------------------+-----------+---------+------------------+----+--------------------+
|   1|Guardians of the ...|     333.13|     76.0|        James Gunn|2014|Guardians of the ...|
|   2|          Prometheus|     126.46|     65.0|      Ridley Scott|2012|    Prometheus_shell|
|   3|               Split|     138.12|     62.0|M. Night Shyamalan|2016|         Split_shell|
+----+--------------------+-----------+---------+------------------+----+--------------------+
only showing top 3 rows



In [124]:
purchaseDf = spark.read.option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/purchases.csv")
purchaseDf.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- apples: integer (nullable = true)
 |-- oranges: integer (nullable = true)



In [125]:
new = purchaseDf.cache()

23/09/22 08:33:54 WARN CacheManager: Asked to cache already cached data.


In [128]:
new.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+

